In [ ]:
from pathlib import Path

# Paste output from benchmark binary. This notebook parses and plots the benchmark results.
benchmark_output_dir = Path("benchmark_output")
system_description = ""
if not list(benchmark_output_dir.glob("*.json")):
    benchmark_output_dir = Path("benchmark_output_example")
    for candidate in sorted(benchmark_output_dir.glob("*.json")):
        system_description = candidate.stem[candidate.stem.find("-")+1:]
        break

save_images_used_in_readme = False
figure_show_mode = 'svg' # 'svg' is inline and visible in GitHub, None is native interactive plotly plots that are not visible in GitHub and several MB in size

In [ ]:
def google_benchmark_output_to_dataframe(path: Path):
    import json

    everything = json.loads(path.read_text())

    sequential = []
    max_p = 1
    # parse out interesting information
    for bench in everything["benchmarks"]:
        # is the benchmark parallel
        bench["parallel"] = "/p:" in bench["name"]

        name_parts = bench["name"].split("/")

        bench["p"] = 1
        for part in name_parts:
            if ":" in part:
                k, v = part.split(':')
                bench[k] = int(v) if k=="p" else v
        max_p = max(max_p, bench["p"])

        if not bench["parallel"]:
            sequential.append(bench)

    for bench in sequential:
        dupe = dict(bench)
        dupe["p"] = max_p
        everything["benchmarks"].append(dupe)

    res = pd.DataFrame(everything["benchmarks"])
    return res

def get_system_description(bench_output: Path):
    import json
    everything = json.loads(bench_output.read_text())
    num_cores = everything.get("context", {}).get("num_cpus", None)
    hostname = everything.get("context", {}).get("host_name", "")

    core_str = f"{num_cores}-Core" if num_cores else ""
    return " ".join([core_str, hostname])


In [ ]:
import pandas as pd
import plotly.express as px

files = []

for candidate in sorted(benchmark_output_dir.glob("*.json")):
    dash = candidate.stem.find("-")
    df = google_benchmark_output_to_dataframe(candidate)
    df.sort_values(by=['impl', 'matrix', 'op', 'lang', 'p'], ascending=[True, False, True, True, True], inplace=True)

    sd = system_description
    if not sd:
        sd = f"{get_system_description(candidate)} - {candidate.name}"

    files.append(dict(
        filename=candidate.stem,
        fn_suffix=candidate.stem[dash+1:],
        fn_prefix=candidate.stem[:dash],
        system_description=sd,
        df=df
    ))


In [ ]:
language_colors = {
    "C++": "#f34b7d",
    "Python": "#3572A5",
}

impl_colors = {
    "IOStream": language_colors["C++"],
    "FMM": "gold",
    "SciPy": language_colors["Python"],
}

def set_consistent_trace_colors(trace):
    for impl, color in impl_colors.items():
        if impl in trace.name:
            trace.update(line=dict(color=color))
    if "Array" in trace.name:
        trace.update(line=dict(dash='dash'))

for file in files:
    for op in "read", "write":
        data = file["df"].copy()
        system_description = file["system_description"]

        data = data[data["op"] == op]
        data["label"] = data.apply(lambda row: f"{row['impl']} {row['matrix']}", axis=1)

        fig = px.line(data, x='p', y='bytes_per_second', title=f"{op.title()} Parallelism ({system_description})", color="label")
        fig.update_layout(
            width=600,
            height=400,
            yaxis = dict(
                title=None,
                ticksuffix = 'B/s',
                rangemode = 'tozero',
                tickformat = '~s'
            ),
            xaxis = dict(title='# of threads'),
            legend=dict(
                title=None,
                # yanchor="top",
                # y=0.99,
                # xanchor="left",
                # x=0.01
            ),
        )

        # Assign consistent trace colors
        fig.for_each_trace(set_consistent_trace_colors)

        fig.show(figure_show_mode)
        if save_images_used_in_readme:
            fig.write_image(f"parallel-scaling-{file['fn_prefix']}-{op}.svg")